In [1]:
import pandas as pd

annotated = pd.read_excel(r"D:\66 CausalMap\Panasonic-IDS\data\MarketIntelligenceReport_Annotation_Tasks.xlsx", engine='openpyxl', sheet_name='MarketIntelligenceReport_Task1')
data = pd.read_csv(r"D:\66 CausalMap\Panasonic-IDS\data\MIR.csv")

In [39]:
df = pd.read_csv(r"D:\66 CausalMap\Panasonic-IDS\data\MarketIntelligenceReport.csv")
df['row_id'] = df.index
cols = ['Translated']

# duplicates, except first
m1 = df.duplicated(cols)
# duplicates including first
m2 = df.duplicated(cols, keep=False)

df['first_occurrence'] = (
 df[cols].merge(df.loc[~m1, cols+['row_id']],how='left')['row_id'].astype(str).where(m1, m2)
)
df

,id,Search date,Geography,Title,Text,Publication date,Original,Translated,Month,Year,Label,row_id,first_occurrence
0,1,27/9/2022 15:22,global,What Makes India The Deathbed Of Global Automa...,"Prior to Ford, two more American firms- Genera...",10/1/2022,en,"Prior to Ford, two more American firms- Genera...",Jan,2022,4,0,False
1,2,27/9/2022 15:22,global,Ford to stop making cars in India,The U.S. carmaker entered India 25 years ago b...,9/9/2021,en,The U.S. carmaker entered India 25 years ago b...,Sep,2021,1,1,True
2,3,27/9/2022 15:22,global,"Ford Motor Company bails on India, will shut c...",Ford is shutting its car factories in India af...,9/9/2021,en,Ford is shutting its car factories in India af...,Sep,2021,1,2,True
3,4,27/9/2022 15:22,global,India approves $3.5 bln scheme to boost clean ...,The government will give about 260 billion rup...,15/9/2021,en,The government will give about 260 billion rup...,Sep,2021,1,3,False
4,5,27/9/2022 15:22,global,More electric vehicles in India soon as automa...,More electric vehicles in India soon as automa...,20/10/2021,en,More electric vehicles in India soon as automa...,Oct,2021,4,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9121,146,22/9/2022 12:27,global,Kebijakan Ketat Covid-19 Bikin Pengusaha Eropa...,"Liputan6.com, Jakarta - Para pengusaha dari Er...",21/9/2022,id,"Liputan6.com, Jakarta - Europeans from Europe ...",Sep,2022,1,9121,True
9122,522,22/9/2022 13:18,china,Kebijakan Ketat Covid-19 Bikin Pengusaha Eropa...,"Liputan6.com, Jakarta - Para pengusaha dari Er...",21/9/2022,id,"Liputan6.com, Jakarta - Europeans from Europe ...",Sep,2022,1,9122,9121
9123,549,22/9/2022 15:37,japan,Certificación EV SSL Participación en el tamañ...,Solicite una copia de muestra del informe:http...,21/9/2022,es,Request a copy of the report: https: //www.mar...,Sep,2022,1,9123,False
9124,550,22/9/2022 15:37,japan,Sistema de gestión térmica de batería EV Dinám...,El informe de investigación de mercado de Sist...,21/9/2022,es,The EV Battery Thermal Management System Marke...,Sep,2022,4,9124,False


In [2]:
import pickle

file = open(r"D:\66 CausalMap\Panasonic-IDS\outs\MIR_rels.obj",'rb')
f_infos = pickle.load(file)
file.close()

In [9]:
from collections import defaultdict

fid2docid = defaultdict(list)
for k,v in f_infos.items():
    for vv in v:
        fid2docid[vv['doc_id']].append(k)

In [31]:
row_id = 51
rowid2docid = {v:k for k,v in enumerate(data['row_id'])}
rowid2docid[row_id]

51

In [64]:
def print_extracted_rels_by_row_id(row_id):
    if row_id not in rowid2docid:
        old = row_id
        row_id = int(df[df['row_id']==row_id]['first_occurrence'].values[0])
        print(f'convert {old}-->{row_id}')
    
    doc_id = rowid2docid[row_id]
    for k in fid2docid[doc_id]:
        print(f_infos[k])
    
    return len(fid2docid[doc_id]) if len(fid2docid[doc_id]) is not None else 0
        
print_extracted_rels_by_row_id(row_id = 7831)

[{'text': 'ASEAN needs to take heed of past experiences in order to be better prepared for future shocks, he said, adding that one key lesson of the pandemic is the importance of a community approach.', 'cause_span': 'order to be better prepared for future shocks', 'effect_span': 'ASEAN needs to take heed of past experiences', 'doc_id': 5484, 'sent_id': 4, 'method': ['unicausal'], 'causenet_patterns': None}]


1

In [66]:
counts = 0
for row_id in annotated['row_id'].unique():
    counts+=print_extracted_rels_by_row_id(row_id)
counts

[{'text': 'It would be hard for Tesla to achieve mass-market volume in India necessary for a venture to make good business sense for the luxury car maker.', 'cause_span': 'It would be hard for Tesla to', 'effect_span': 'necessary for a venture to make good business sense for the luxury car maker .', 'doc_id': 51, 'sent_id': 2, 'method': ['unicausal'], 'causenet_patterns': None}]
[{'text': 'Foreign or local electric cars (EVs) will grow in India only when there\'s price parity with its internal combustion engine (ICE) equivalent. "', 'cause_span': 'there \' s price parity with its internal combustion engine ( ICE ) equivalent . "', 'effect_span': 'Foreign or local electric cars ( EVs ) will grow in India', 'doc_id': 51, 'sent_id': 3, 'method': ['unicausal'], 'causenet_patterns': None}]
[{'text': '"Given these conditions, it is understandable that local as well as foreign players may hesitate to expand in the Indian market. "', 'cause_span': 'these conditions', 'effect_span': ', it is un

23